In [ ]:
import civis
import os
import requests
import json
import pandas as pd
import urllib.parse
import time
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
TABLES = ['Affiliate','AffiliatePerCapita','AffiliateDivision','Division','Accounts','StateFederation','EntityType','Geographic State','AffiliateType','FiduciaryBondCoverage','AffiliateDesignation','AffiliateInactiveReason','AffiliateGeoReach', 'Region']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']

            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower():item['key']})
                self.APP_DICT.update({name.lower():{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']

    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_dual_connection(self,obj_name,field_name1,connection1,field_name2,connection2):
        #Convert to IDs
        knack_object_id = self.APP_DICT[obj_name]['obj_id']
        knack_field_id1 = self.APP_DICT[obj_name]['fields'][field_name1]
        knack_field_id2 = self.APP_DICT[obj_name]['fields'][field_name2]

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':knack_field_id1, 'operator':'is', 'value': connection1},{'field':knack_field_id2, 'operator':'is', 'value': connection2}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''
    
    def get_connection(self,obj_name,field_name,connection):
        time.sleep(1)
        #Convert to IDs
        knack_object_id = self.APP_DICT[obj_name]['obj_id']
        knack_field_id = self.APP_DICT[obj_name]['fields'][field_name]

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':knack_field_id, 'operator':'is', 'value': connection}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''

client = KnackAFT()

def add_affiliate(record):
    aff_dict = client.APP_DICT['affiliate']['fields']
    connection_fields = ["retireeentitytypeid", "parentaffiliateid","affiliatetypeid","affiliatedesignationid","affiliategeoreachid","affiliateinactivereasonid", "locationstateabr", "regionid"]
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateinactivereasonid':
                conn = client.get_connection('affiliateinactivereason','affiliateinactivereasonid',v)
                out.update({aff_dict['affiliateinactivereason']:conn})
            elif k == 'parentaffiliateid':
                conn = client.get_connection('statefederation','affiliateid',v)
                out.update({aff_dict[k]:conn})
            elif k == 'locationstateabr':
                conn = client.get_connection('geographic state', 'geographic state name', v)
                out.update({aff_dict[k]:conn})
            elif k == 'retireeentitytypeid':
                conn = client.get_connection('entitytype', 'entitytypeid', v)
                out.update({aff_dict[k]:conn})
            else:
                conn = client.get_connection(k.replace('id',''),k,v)
                out.update({aff_dict[k]:conn})

        else:
            out.update({aff_dict[k]:v})

    out.update({aff_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliate', 'affiliateid', record['affiliateid'])
    if check_exists:
        print(record['affiliateid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliate with id: " + record['affiliateid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
        
        
def add_affiliatepercapita(record):
    record.pop('includestatepercapita')
    apc_dict = client.APP_DICT['affiliatepercapita']['fields']
    connection_fields = ['affiliateid', 'currentfiduciarybondcoverageid']
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({apc_dict['affiliate']:conn})
            else:
                conn = client.get_connection('fiduciarybondcoverage','fiduciarybondcoverageid',v)
                out.update({apc_dict['fiduciarybondcoverage']:conn})

        else:
            out.update({apc_dict[k]:v})

    if not out['field_658']:
        out.update({'field_658':'DONTSENDINVOICE'})
        
    out.update({apc_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliatepercapita', 'affiliatepercapitaid', record['affiliatepercapitaid'])
    if check_exists:
        print(record['affiliatepercapitaid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliatepercapita with id: " + record['affiliatepercapitaid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
        
def add_affiliatedivision(record):
    adiv_dict = client.APP_DICT['affiliatedivision']['fields']
    connection_fields = ["affiliateid","divisionid"]
    out = {}
    aff_conn = ''
    div_conn = ''
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                aff_conn = client.get_connection('affiliate','affiliateid',v)
                out.update({adiv_dict['affiliate']:aff_conn})
            elif k == 'divisionid':
                div_conn = client.get_connection('division','divisionid',v)
                out.update({adiv_dict['division']:div_conn})
        else:
            out.update({adiv_dict[k]:v})

    out.update({adiv_dict['aftdbupdate']:1})

    check_exists = client.get_dual_connection('affiliatedivision', 'affiliate', aff_conn, 'division', div_conn)
    if check_exists:
        print("Affiliate:"+record['affiliateid']+" and Division:"+record['divisionid']+ " found! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatedivision"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliatedivision with id: " + record['affiliatedivisionid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatedivision"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))

def run_affiliate():
    table_name = os.environ['civis_table_path'] + 'sys_arts_affiliateuploader_xn'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)

        df['updatedat'] = pd.to_datetime(df.updatedat)
        df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
        
        df['createdat'] = pd.to_datetime(df.createdat)
        df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
        
        df['deletedat'] = pd.to_datetime(df.deletedat)
        df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

        aff_dict = client.APP_DICT['affiliate']['fields']
        aff_dict.update({
            'affiliateein':aff_dict.pop('affiliate ein'),
            'affiliatepercapitapin':aff_dict.pop('affiliate percapita pin'),
            'ischartered':aff_dict.pop('is affiliate chartered'),
            'charterdate':aff_dict.pop('charter date'),
            'parentaffiliateid':aff_dict.pop('state federation'),
            'affiliatetypeid':aff_dict.pop('affiliate type'),
            'affiliatedesignationid':aff_dict.pop('affiliate designation'),
            'affiliategeoreachid':aff_dict.pop('affiliate geographical reach'),
            'locationstateabr':aff_dict.pop('location state'),
            'affiliateguid':aff_dict.pop('affiliate guid'),
            'affiliatenumber':aff_dict.pop('affiliate number'),
            'affiliateabbreviatedname':aff_dict.pop('affiliate abbreviation'),
            'affiliateacronym':aff_dict.pop('affiliate acronym'),
            'isaffiliateactive':aff_dict.pop('is affiliate active'),
            'affiliateinactivereason':aff_dict.pop('affiliate inactive reason'),
            'affiliateinactivedate':aff_dict.pop('affiliate inactive date'),
            'regionid':aff_dict.pop('region'),
        })
        for payload_dict in df.to_dict('records'):
            add_affiliate(payload_dict)
                
def run_affiliatepercapita():
    table_name = os.environ['civis_table_path'] + 'sys_arts_affiliatepercapitauploader_xn'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)
        
        df['updatedat'] = pd.to_datetime(df.updatedat)
        df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
        
        df['createdat'] = pd.to_datetime(df.createdat)
        df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
        
        df['deletedat'] = pd.to_datetime(df.deletedat)
        df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

        for payload_dict in df.to_dict('records'):
            add_affiliatepercapita(payload_dict)

def run_affiliatedivision():
    table_name = os.environ['civis_table_path'] + 'sys_arts_affiliatedivisionuploader_xn'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)
        
        df['updatedat'] = pd.to_datetime(df.updatedat)
        df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
        
        df['createdat'] = pd.to_datetime(df.createdat)
        df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
        
        df['deletedat'] = pd.to_datetime(df.deletedat)
        df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

        for payload_dict in df.to_dict('records'):
            add_affiliatedivision(payload_dict)


# print("Uploading affiliate...")
# run_affiliate()
# print("--------------------------------")

print("Uploading affiliatepercapita...")
run_affiliatepercapita()
print("--------------------------------")

print("Uploading affiliatedivision...")
run_affiliatedivision()
print("--------------------------------")
